In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp data

# Data

> Utilities for retrieving benchmarks and generating random graphs

In [ ]:
#export 
from nbdev.showdoc import *
import numpy as np
import networkx as nx
import scipy

## Benchmarks

In [ ]:
def cora():
    cora = np.load('data/cora_gnnbench.npz', allow_pickle=True)
    A, X, y = cora['A'], cora['X'], cora['y']
    return A, X, y

## Synthetic

In [ ]:
#export
def make_planar_graph(n):
    """
    Makes a planar graph with n nodes
    
    Code adapted from https://stackoverflow.com/questions/26681899/how-to-make-networkx-graph-from-delaunay-preserving-attributes-of-the-input-node
    """
    points = np.random.rand(n, 2)
    delTri = scipy.spatial.Delaunay(points)
    edges = set()
    for n in range(delTri.nsimplex):
        edge = sorted([delTri.vertices[n,0], delTri.vertices[n,1]])
        edges.add((edge[0], edge[1]))
        edge = sorted([delTri.vertices[n,0], delTri.vertices[n,2]])
        edges.add((edge[0], edge[1]))
        edge = sorted([delTri.vertices[n,1], delTri.vertices[n,2]])
        edges.add((edge[0], edge[1]))
    graph = nx.Graph(list(edges))
    pos = pos = dict(zip(range(len(points)), points))
    return graph, pos

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_graphtools.ipynb.
Converted 01_sampling.ipynb.
Converted 02_spectral.ipynb.
Converted 03_perturb.ipynb.
Converted 04_plotting.ipynb.
Converted 05_data.ipynb.
Converted index.ipynb.
